In [ ]:
# standard import of pandas
import pandas as pd

# numpy, "numerical python" - we'll cover this in the following notebooks.
import numpy as np

# hides warning messages
import warnings
warnings.filterwarnings("ignore")
# additional import of the geopandas package
import geopandas as gpd 

import zipfile # package for unzipping zip files

import fiona; #help(fiona.open) # driver for geopandas 

from shapely.geometry import Point, Polygon

import matplotlib.pyplot as plt

In [ ]:
anlagebaeume = gpd.read_file("data/cleaned/anlage_cl.geojson")
strassenbaeume = gpd.read_file("data/cleaned/strasse_cl.geojson")


In [ ]:
lor = gpd.read_file("data/LOR_Planungraeume_Berlin.geojson")

In [ ]:
#looking at lor, deciding which columns to keep
lor

In [ ]:
columns_to_keep_lor = ['OBJECTID', 'PLR_ID', 'PLR_NAME', 'BEZ', 'BZR_NAME', 'FINHALT', 'geometry']


In [ ]:
lor_cl = lor[columns_to_keep_lor]

In [ ]:
lor_cl.rename(columns={"OBJECTID": "lor_id"}, inplace=True)

In [ ]:
lor_cl.head()

In [ ]:
lor_cl.info() # we still need to convert PLR ID and BEZ to INT

In [ ]:
#lowercase for column headers
lor_cl= lor_cl.rename(columns=str.lower)

In [ ]:
lor_cl.head()


In [ ]:
#converting plr_id from object to int (it worked on the first try)
lor_cl['plr_id'] = lor_cl['plr_id'].astype(int)

In [ ]:
#same for bez, frist try worked. omg
lor_cl['bez'] = lor_cl['bez'].astype(int)

In [ ]:
#info 
lor_cl.info()

In [ ]:
lor_cl.to_file("data/cleaned/lor_cl.geojson", driver='GeoJSON')

In [ ]:
#preparing testplot 
#crs
strassenbaeume = strassenbaeume.to_crs(lor_cl.crs)

In [ ]:
# testplot to see if geodata still intact and OBJECT ID labels are working
lor_cl['coords'] = lor_cl['geometry'].apply(lambda x: x.representative_point().coords[:])
lor_cl['coords'] = [coords[0] for coords in lor_cl['coords']]

fig, ax = plt.subplots(figsize=(25,25))
ax.set_aspect('equal')
lor_cl.boundary.plot(ax=ax, color='black', edgecolor='black')
strassenbaeume.plot(ax=ax, linewidth=1,marker = 'o', color="seagreen", markersize=5)
lor_cl.apply(lambda x: ax.annotate(text=x['lor_id'], xy=x.geometry.centroid.coords[0], ha='center'), axis=1)

plt.show();

#YAY it works "FREE PIZZA FOR EVERYONE"

In [ ]:
# preparing to cluster 
# anlagebaeume & lor_cl --> geojoson anlagebaeume --> point; lor_cl --> poly  
#mapping crs of anlagebaeume to crs of lor_cl
anlagebaeume = anlagebaeume.to_crs(lor_cl.crs)

In [ ]:
#Perform spatial join to match points --> 'anlagebaeume' and polygons --> 'lor'
anlage_sj = gpd.tools.sjoin(anlagebaeume, lor_cl, predicate="within", how='left')

In [ ]:
#check table structure
anlage_sj

In [ ]:
ids_to_keep = ['baumid', 'lor_id', 'geometry']
anlage_sj_rdcd = anlage_sj[ids_to_keep]
strasse_sj_rdcd = strasse_sj[ids_to_keep]

In [ ]:
display(anlage_sj_rdcd)
display(strasse_sj_rdcd)

In [ ]:
anlage_sj_rdcd.to_file("data/cleaned/anlage_ids_clean.geojson", driver='GeoJSON')

In [ ]:
strasse_sj_rdcd.to_file("data/cleaned/strasse_ids_clean.geojson", driver='GeoJSON')

In [ ]:
#Perform spatial join to match points --> 'strassenbaeume' and polygons --> 'lor_cl'
strasse_sj = gpd.tools.sjoin(strassenbaeume, lor_cl, predicate="within", how='left')

In [ ]:
#check table structure
strasse_sj.head(1)

In [ ]:
#clustering all 'strassenbaeume' in LOR 525 Parchimer Allee Süd
str_clstr_525 = strassenbaeume[strasse_sj.lor_id==525]

In [ ]:
str_clstr_525

In [ ]:
# testplot to see if geodata still intact 
lor_cl['coords'] = lor_cl['geometry'].apply(lambda x: x.representative_point().coords[:])
lor_cl['coords'] = [coords[0] for coords in lor_cl['coords']]

fig, ax = plt.subplots(figsize=(25,25))
ax.set_aspect('equal')
lor_cl.boundary.plot(ax=ax, color='black', edgecolor='black')
strassenbaeume.plot(ax=ax, linewidth=1,marker = 'o', color="seagreen", markersize=5)
str_clstr_525.plot(ax=ax, marker='o', color='gold', markersize=5)
lor_cl.apply(lambda x: ax.annotate(text=x['lor_id'], xy=x.geometry.centroid.coords[0], ha='center'), axis=1)

plt.show();

#YAY it works "FREE PIZZA FOR EVERYONE"

In [ ]:
# read csv with LOR id and LOR name as lor_ids
lor_fc = pd.read_csv('data/lor_id.csv')

In [ ]:
#create list of all LOR IDs to perform for loop
lor_lst = lor_fc['lor_id'].tolist()

In [ ]:
#checking new files

boden_berlin = gpd.read_file("data/Bodenkundliche Kennwerte.geojson")

In [ ]:
boden_berlin.head()

In [ ]:
vers_berlin = gpd.read_file("data/Versiegelung.geojson")

In [ ]:
#keep 'vg', 'vgstrasse', 'feld_30',	'feld_150', 'geometry' 'regenja' 'regenso'
vers_berlin.head()
#schl5 --> ID --> rename to boden_id

In [ ]:
#  31  phobermin      24655 non-null  float64 
# 32  phobermax      24655 non-null  object  
# 33  phoberdur      24655 non-null  float64 
# 34  phstufe_o      24655 non-null  float64 
# 35  phstufe_o_bez  24655 non-null  object  
# 36  phuntermin     24655 non-null  object  
# 37  phuntermax     24655 non-null  object  
# 38  phunterdur     24655 non-null  float64 
# 39  phstufe_u      24655 non-null  float64 
# 40  phstufe_u_bez  24655 non-null  object  
# 54  kf_o           24655 non-null  float64 
# 55  kfstufe_o      24655 non-null  float64 
# 56  kfstufe_o_bez  24655 non-null  object  
# 57  kf_u           24655 non-null  float64 
# 58  kfstufe_u      24655 non-null  float64 
# 59  kfstufe_u_bez  24655 non-null  object  
# 60  kf             24655 non-null  float64 
# 61  kfstufe        24655 non-null  float64 
# 62  kfstufe_bez    24655 non-null  object  
# 63  geometry       24655 non-null  geometry
boden_berlin.info()

#schl5 --> ID --> rename to boden_id